In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

In [ ]:
def add_s_layers(river_ds):
    # Creating new array to fill the gaps
    v_data = np.zeros((5, 175))
    v_add = xr.DataArray(v_data, dims=river_ds['river_Vshape'].dims, coords=river_ds['river_Vshape'].coords)
    temp_data = np.ones((4019, 5, 175)) * 5
    t_add = xr.DataArray(temp_data, dims=river_ds['river_temp'].dims, coords=river_ds['river_temp'].coords)
    salt_data = np.ones((4019, 5, 175))
    s_add = xr.DataArray(salt_data, dims=river_ds['river_salt'].dims, coords=river_ds['river_salt'].coords)
    
    # New ready data arrays
    river_Vshape = xr.concat([v_add, river_ds['river_Vshape']], dim="s_rho")
    river_temp = xr.concat([t_add, river_ds['river_temp']], dim="s_rho")
    river_salt = xr.concat([s_add, river_ds['river_salt']], dim="s_rho")
    
    # Remove old and add new variables with new s_rho=40 dimension
    river_ds = river_ds.drop_dims('s_rho')
    river_ds = river_ds.assign(river_Vshape=river_Vshape, river_temp=river_temp, river_salt=river_salt)

    return river_ds

In [ ]:
def mask_rivers(ds):
    ds['river_transport'].values = np.zeros((4019, 175))
    return ds

In [ ]:
def check_rivers_runoff(ds, verbose=False):
    xi_eta_dir = []
    for i in range(ds.dims['river']):
        river = ds.isel(river=i, river_time=0)
        cell_runoff = (abs(river.river_transport) * river.river_Vshape).max().values
        if cell_runoff > 10:
            if verbose:
                print(f"River {i+1} max runoff: {cell_runoff}")
                print(f"Coordinates: {river.river_Xposition}; {river.river_Eposition}")
                print(f"Direction: {river.river_direction} \n")
            xi_eta_dir.append((int(river.river_Xposition.values), 
                               int(river.river_Eposition.values),
                               int(river.river_direction.values)
                               ))
    return xi_eta_dir

In [ ]:
def save_rivers(ds, name):
    ds.to_netcdf(path='fram_data/norfjords_160m_river_{name}.nc', format='NETCDF4')

Check rivers

In [ ]:
ds = xr.open_dataset('fram_data/norfjords_160m_river_v2.nc')
grid_ds = xr.open_dataset('fram_data/norfjords_160m_grid_version2+.nc')
xi_eta_dir = check_rivers_runoff(ds)

In [ ]:
def plot_river_estuary_u(river_xi, river_eta, river_dir):

    assert river_dir == 0
    plt.figure(figsize=(15, 4))

    plt.subplot(121)
    grid_ds.mask_u.isel(
        xi_u=slice(river_xi - 2, river_xi + 3), 
        eta_u=slice(river_eta - 2, river_eta + 3)
        ).plot()

    grid_ds.mask_u[dict(xi_u=river_xi, eta_u=river_eta)] = np.nan

    plt.subplot(122)
    grid_ds.mask_u.isel(
        xi_u=slice(river_xi - 2, river_xi + 3), 
        eta_u=slice(river_eta - 2, river_eta + 3)
        ).plot()

    plt.suptitle(f"Xi: {river_xi}; Eta: {river_eta}; Direction: {river_dir}.")


def plot_river_estuary_v(river_xi, river_eta, river_dir):

    assert river_dir == 1
    plt.figure(figsize=(15, 4))

    plt.subplot(121)
    grid_ds.mask_v.isel(
        xi_v=slice(river_xi - 2, river_xi + 3), 
        eta_v=slice(river_eta - 2, river_eta + 3)
        ).plot()

    grid_ds.mask_v[dict(xi_v=river_xi, eta_v=river_eta)] = np.nan

    plt.subplot(122)
    grid_ds.mask_v.isel(
        xi_v=slice(river_xi - 2, river_xi + 3), 
        eta_v=slice(river_eta - 2, river_eta + 3)
        ).plot()

    plt.suptitle(f"Xi: {river_xi}; Eta: {river_eta}; Direction: {river_dir}.")

In [ ]:
"""
Probably, not sure: Due to roms notation where rho points start from zero 
(so not such points in fortran), eta coordinate in mask_u already "has"
pythonic coordinates.
https://www.myroms.org/wiki/Grid_Generation
"""
for xi, eta, direction in xi_eta_dir:
    if direction == 0: 
        xi, eta = xi - 1, eta
        plot_river_estuary_u(xi, eta, direction)
    else:
        xi, eta = xi, eta - 1
        plot_river_estuary_v(xi, eta, direction)